## Import relevant libraries

In [1]:
import requests
import re
import time
import numpy as np
import pickle

from bs4 import BeautifulSoup

import pandas as pd
import string
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

## Build a function that exctracts all lyrics from an artist and store it in a list

chosen/needed url's:https://www.lyrics.com/artist/Lana-Del-Rey/2487752
                    https://www.lyrics.com/artist/a-ha/3491
                    https://www.lyrics.com/artist/blink-182/211247

In [2]:
def get_lyrics (url):
    
    """
    - get artist page from links: artist_links -> artist_pages
    - extract links from the artist page: artist_pages -> artist_song_links
    - downloads links from the song page: song_links -> song_pages
    - extract lyrcis from the song page: song_pages -> lyrics_list
    - cleans the lyrics, returns a list of lyrics: lyrics_list -> clean_lyrics_list
    """
    r = requests.get(url)
    regex = r'a\shref="(\/lyric\/.+?)"'
    matches = re.findall(regex, r.text)

    song_links = []

    for i in matches:
        full_link = "http://www.lyrics.com"+ i
        song_links.append(full_link)

    lyric_html = []

    for song_link in song_links: #here we can define how much links we want to extract after song_links[]
        time.sleep(1)
        resp = requests.get(song_link)
        lyric_html.append(resp.text)
     
    lyrics = []

    for lyric_html_string in lyric_html:
        soup = BeautifulSoup(lyric_html_string, "html.parser")
        #lyrics_raw.append(soup)
        songtext = soup.select("#lyric-body-text")
        songtext = songtext[0].get_text()
        lyrics.append(songtext) 
         
    final_artist_lyric_list = []
    replace_items =[".", ";", "—", "_", '“', ":", "\n", "(", ")", "?", "\\",",", "()", "'", ","]
    
    
    for lyric in lyrics:
        lyric_lowercase = lyric.lower().strip()
        
    
        for item in replace_items:
            lyric_lowercase = lyric_lowercase.replace(item, " ")
        final_artist_lyric_list.append(lyric_lowercase)
    
    return final_artist_lyric_list
        
#pac = get_lyrics("https://www.lyrics.com/artist/2Pac/50051")

In [3]:
#print(len(pac))

## Convert lyrics to PandaDataFrame, save as CSV and add Artist name

In [4]:
def convert_to_pddf (lyrics_from_artist, artistname, filename):
    data = {"songtext": lyrics_from_artist, "artist": artistname}
    lyrics = pd.DataFrame(data)
    lyrics.to_csv(filename)
    return lyrics

#convert_to_pddf(pac, "2pac", "2pac.csv")


In [5]:
df1 = pd.read_csv('data/a_ha.csv', index_col=[0])
df2=pd.read_csv('data/blink182.csv', index_col=[0])
df3=pd.read_csv('data/lanadelrey.csv', index_col=[0])
df4=pd.read_csv("data/2pac.csv", index_col=[0])

df1 = df1.append(df2)
df1 = df1.append(df3)
df1 = df1.append(df4)

all_artists_lyrics = df1[["artist","songtext"]]
all_artists_lyrics.head(5)

,artist,songtext
0,A_ha,we re talking away\r i don t know what\r i m t...
1,A_ha,we re talking away i don t know what i m to sa...
2,A_ha,sometimes the way we talk isn t all that good ...
3,A_ha,hold me tight this is a lonely night and i ve ...
4,A_ha,i ll never let you see the way my broken heart...


In [6]:
all_artists_lyrics.artist.value_counts()

2pac            1242
A_ha            1189
Blink182         930
Lana_del_Rey     514
Name: artist, dtype: int64

In [7]:
#all_artists_lyrics["artist_label"] = all_artists_lyrics.artist.map({"A_ha": 0, "Blink182": 1,"Lana_del_Rey": 2})

#all_artists_lyrics.head(3)

In [8]:
X = all_artists_lyrics.songtext
y = all_artists_lyrics.artist   # store the feature matrix (X) and response vector (y)

print(X.shape)                  # check the shapes of X and y
print(y.shape)

(3875,)
(3875,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2906,)
(969,)
(2906,)
(969,)


In [10]:
vect = CountVectorizer( stop_words='english',max_df=0.90,min_df=2 ) 
# import and instantiate CountVectorizer (with the default parameters)

In [11]:
vect.fit(X_train)   # learn the 'vocabulary' of the training data

vect.get_feature_names()      # examine the fitted vocabulary

X_train_dtm = vect.transform(X_train)  # transform training data into a 'document-term matrix'
X_train_dtm               

<2906x9010 sparse matrix of type '<class 'numpy.int64'>'
	with 276749 stored elements in Compressed Sparse Row format>

In [12]:
X_train_dtm.toarray()       #convert sparse matrix to a dense matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())  #examine the vocabulary and document-term matrix together

,04,10,106,11,12,122nd,125,13,14,15,...,zero,zig,zip,zipped,zodiac,zone,zones,zoo,zow,zulu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2902,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X_test_dtm = vect.transform(X_test)   # transform test data into a 'document-term matrix'
X_test_dtm

<969x9010 sparse matrix of type '<class 'numpy.int64'>'
	with 91879 stored elements in Compressed Sparse Row format>

## Define Model (The multinomial Naive Bayes classifier is suitable for classification with discrete features.)

In [15]:
nb = MultinomialNB
nb = MultinomialNB()

In [16]:
%time nb.fit(X_train_dtm, y_train)        # train the model using X_train_dtm 


Wall time: 1 s


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
y_pred_class = nb.predict(X_test_dtm)     # make class predictions for X_test_dtm

## Save models on harddisk with pickle

In [18]:
with open('model.pickle', 'wb') as f:
    pickle.dump(nb, f)
    
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vect, f)

In [19]:
metrics.accuracy_score(y_test, y_pred_class)   # calculate accuracy of class predictions

0.9803921568627451

In [20]:
metrics.confusion_matrix(y_test, y_pred_class)      # print the confusion matrix


array([[310,   1,   1,   2],
       [  2, 318,   1,   1],
       [  3,   3, 209,   2],
       [  0,   3,   0, 113]], dtype=int64)

testtext = ["I was walking in the rain"]        

testtext_dtm = vect.transform(testtext)
predicttt = nb.predict_proba(testtext_dtm)

print(predicttt)

### Build input function for user

In [21]:
testtext = input("Please type in any text: ")

testtext_dtm = vect.transform([testtext])
which_artist = nb.predict(testtext_dtm)
prob_artist = nb.predict_proba(testtext_dtm)

print(which_artist)
print(prob_artist)

print(" This lyric snatch fits best to the artist " + str(which_artist) + " with a probability of " +  str(max(prob_artist[0])))

Please type in any text: I need a smoke
['2pac']
[[0.76163326 0.00898368 0.16171476 0.0676683 ]]
 This lyric snatch fits best to the artist ['2pac'] with a probability of 0.7616332588665898
